In [ ]:
#dont run this here
#this is the colab notebook link right here B)

# https://colab.research.google.com/drive/1ghvsRoDjdouULrFTD0iF2_V-IUuExauI?usp=sharing

# just run the second last block, then the last block to see the plot

In [1]:
!pip install pathway bokeh --quiet

In [2]:
import pandas as pd
import numpy as np
import datetime
import pathway as pw
import bokeh.plotting
import panel as pnl

In [3]:
#importing the data
# https://medium.com/data-science/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

url = 'https://raw.githubusercontent.com/monarchxr/Summer-Analytics-IITG/refs/heads/main/Week5/Modified%20-%20modified.csv'
df = pd.read_csv(url)
df

,Unnamed: 0,SystemCodeNumber,Capacity,Occupancy,LastUpdatedDate,LastUpdatedTime,IsSpecialDay,VehicleType,Latitude,Longitude,TrafficConditionNearby,QueueLength
0,0,BHMBCCMKT01,577,61,04-10-2016,07:59:42,0,car,28.5,77.15,low,2
1,1,BHMBCCMKT01,577,64,04-10-2016,08:25:42,0,car,28.5,77.15,average,2
2,2,BHMBCCMKT01,577,80,04-10-2016,08:59:42,0,car,28.5,77.15,low,2
3,3,BHMBCCMKT01,577,107,04-10-2016,09:32:46,0,car,28.5,77.15,low,3
4,4,BHMBCCMKT01,577,150,04-10-2016,09:59:48,0,car,28.5,77.15,low,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1307,1307,BHMBCCMKT01,577,309,19-12-2016,14:30:33,0,bike,28.5,77.15,average,5
1308,1308,BHMBCCMKT01,577,300,19-12-2016,15:03:34,0,car,28.5,77.15,low,4
1309,1309,BHMBCCMKT01,577,274,19-12-2016,15:29:33,0,truck,28.5,77.15,low,3
1310,1310,BHMBCCMKT01,577,230,19-12-2016,16:03:35,0,cycle,28.5,77.15,low,2


In [4]:
# df.info()
df.describe()

,Unnamed: 0,Capacity,Occupancy,IsSpecialDay,Latitude,Longitude,QueueLength
count,1312.000000,1312.0,1312.000000,1312.000000,1312.0,1.312000e+03,1312.000000
mean,655.500000,577.0,162.029726,0.150915,28.5,7.715000e+01,4.978659
std,378.886087,0.0,99.404980,0.358102,0.0,1.393195e-12,3.040209
min,0.000000,577.0,2.000000,0.000000,28.5,7.715000e+01,0.000000
25%,327.750000,577.0,91.000000,0.000000,28.5,7.715000e+01,2.000000
50%,655.500000,577.0,157.000000,0.000000,28.5,7.715000e+01,4.000000
75%,983.250000,577.0,206.000000,0.000000,28.5,7.715000e+01,7.000000
max,1311.000000,577.0,573.000000,1.000000,28.5,7.715000e+01,15.000000


In [5]:
df['LastUpdatedTime'] = pd.to_datetime(df['LastUpdatedTime'])

df['delta'] = df['LastUpdatedTime'].diff()
print(df['delta'].value_counts().head(10))

delta
0 days 00:33:00    78
0 days 00:27:00    67
0 days 00:27:01    64
0 days 00:33:01    57
0 days 00:32:59    48
0 days 00:27:02    46
0 days 00:26:58    46
0 days 00:33:02    43
0 days 00:26:00    41
0 days 00:34:01    40
Name: count, dtype: int64


/tmp/ipython-input-5-2707220035.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['LastUpdatedTime'] = pd.to_datetime(df['LastUpdatedTime'])


In [6]:
# Combine date + time
df['LastUpdatedDate'] = pd.to_datetime(df['LastUpdatedDate'], format='%d-%m-%Y')
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'].dt.strftime('%d-%m-%Y') + ' ' + df['LastUpdatedTime'].dt.strftime('%H:%M:%S'),
                                  format='%d-%m-%Y %H:%M:%S')

# Sort and round
df = df.sort_values('Timestamp').reset_index(drop=True)
df['rounded_time'] = df['Timestamp'].dt.round('30min')

In [7]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [8]:
class ParkSchema(pw.Schema):
  Timestamp: str
  Occupancy: int
  Capacity: int

In [9]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkSchema, input_rate=1000)

In [10]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [11]:
# For now, we'll simulate previous price = 10 (start of each day)
@pw.udf
def get_base_price(_):
    return 10.0

@pw.udf
def linear_price_model(prev_price, occupancy, capacity):
    if capacity == 0:
        return prev_price

    ratio = occupancy / capacity

    if ratio < 0.3:
        alpha = 0.5
    elif ratio < 0.7:
        alpha = 1.5
    else:
        alpha = 3.0

    return prev_price + alpha * ratio

In [12]:
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.Occupancy),
        cap_max = pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        # base = get_base_price(pw.this.t),
        price = linear_price_model(10, pw.this.occ_max, pw.this.cap_max)
    )
)

In [15]:
# Activate the Panel extension to enable interactive visualizations
pnl.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pnl.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [16]:
%%capture --no-display
pw.run()

Output()